In [8]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
import time
from langchain.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from difflib import SequenceMatcher
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import re
from difflib import SequenceMatcher
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chains import LLMChain
os.environ["OPENAI_API_KEY"] = "sk-"
embeddings = OpenAIEmbeddings()
from langchain.docstore.document import Document

In [9]:
def create_progress_file(user_id):
    folder_path = f"progress/{user_id}"
    file_path = f"{folder_path}/progress.txt"
    file_path2 = f"{folder_path}/questions.txt"
    

    # Check if folder exists, create it if it doesn't
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    # Check if file exists, create it if it doesn't
    if not os.path.exists(file_path):
        with open(file_path, 'a') as progress_file:
            pass
    if not os.path.exists(file_path2):   
        with open(file_path2, 'w') as progress_file:
            progress_file.write('0')

def fix_collection_name(collection_name):
    # Replace spaces with underscores
    collection_name = collection_name.replace(' ', '_')

    # Add underscore placeholders if the name is too short
    if len(collection_name) < 3:
        collection_name += '_' * (3 - len(collection_name))

    # Cut off the name if it is too long
    if len(collection_name) > 63:
        collection_name = collection_name[:63]

    # Replace consecutive periods with a single period
    collection_name = re.sub(r'\.\.', '.', collection_name)

    # Replace any invalid characters with underscores
    collection_name = re.sub(r'[^a-zA-Z0-9_.-]', '_', collection_name)

    # Ensure that the name starts and ends with an alphanumeric character
    if not collection_name[0].isalnum():
        collection_name = 'X' + collection_name[1:]
    if not collection_name[-1].isalnum():
        collection_name = collection_name[:-1] + 'X'

    return collection_name

with open("indexholders.txt", "a", encoding="utf-8") as f:
    pass

def read_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)


def folder_exists(path):
    return os.path.isdir(path)

def create_progress_folder(user_id,phase):
    folder_path = os.path.join("progress", user_id)
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, "progress.txt")
    with open(file_path, "w") as progress_file:
        progress_file.write(phase)

    
def check_similarity(string1, string2):
    # Convert strings to lowercase to ignore casing
    string1 = string1.lower()
    string2 = string2.lower()

    # Calculate similarity ratio
    similarity_ratio = SequenceMatcher(None, string1, string2).ratio()

    # Check if similarity probability is greater than or equal to 70%
    if similarity_ratio >= 0.7:
        return True
    else:
        return False


def remove_first_line(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    # Exclude the first line
    updated_lines = lines[1:]

    with open(file_path, "w") as file:
        file.writelines(updated_lines)

        


def has_similar_word(target_word, word_list):
    threshold = 0.7
    
    for word in word_list:
        similarity = SequenceMatcher(None, target_word, word).ratio()
        if similarity >= threshold:
            return True
    
    return False




In [10]:
def chat(query,userid):
    vectorstore_folder = f'db/{userid}/fullchat'
    vectorstore_folder2 = f'db/{userid}/assessment'
    vectorstore_folder3 = f'db/{userid}/homeworkdb'
    vectorstore_folder4 = f'db/{userid}/goalsdb'
    
    users = read_file('indexholders.txt')
    if userid not in users:
        vectordb = Chroma(collection_name=f'x{userid}',persist_directory=vectorstore_folder, embedding_function=embeddings)
        goalsdb = Chroma(collection_name=f'x{userid}goalsdb',persist_directory=vectorstore_folder4, embedding_function=embeddings)
        assessmentdb = Chroma(collection_name=f'x{userid}assessment',persist_directory=vectorstore_folder2, embedding_function=embeddings)
        homeworkdb = Chroma(collection_name=f'x{userid}homework',persist_directory=vectorstore_folder3, embedding_function=embeddings)
        
        Chroma.add_texts(goalsdb,texts=["goals"])
        Chroma.add_texts(vectordb,texts=["history"])
        Chroma.add_texts(assessmentdb,texts=["Problem Assessment"])
        Chroma.add_texts(homeworkdb,texts=["HOMEWORK"])
        

        vectordb.persist()
        goalsdb.persist()
        assessmentdb.persist()
        homeworkdb.persist()
        vectordb = None
        goalsdb = None
        assessmentdb = None
        homeworkdb = None

    filepath2 = f"progress/{userid}/buffer.txt"
    buffer = ''
    if os.path.exists(filepath2):   
        with open(filepath2, 'r') as progress_file:
            buffer = progress_file.read()
            
    current_timestamp = time.time()
    formatted_timestamp = time.strftime("%Y-%m-%d", time.localtime(current_timestamp))
    userid = fix_collection_name(userid.lower())
    filepath = f'progress/{userid}/progress.txt'
    create_progress_file(userid)
    response = ''
    #STAGE DETECTION
    ExpectedStage = ''
    with open(filepath, 'r') as progress_file:
        ExpectedStage = progress_file.readline().strip()
        
    ExpectedStage =  analyzer_bot2(query,buffer,userid,ExpectedStage)
    
    
    #IF EMPTY THEN ITS INTRODUCTION PHASE
    if ExpectedStage == '' or check_similarity(ExpectedStage,"Introduction"):
        response = introduction_bot(query,formatted_timestamp,userid)
    
    #THIS IS HANDELING THE ASSESSMENT PHASE
    elif check_similarity(ExpectedStage,'ASSESSMENT'):
        response = assessment_bot(query,formatted_timestamp,userid)
    
    #THIS IS HANDELING THE GOALS SETTING PHASE    
    elif check_similarity(ExpectedStage,'GOALS SETTING'):
        response = goal_setter(query,userid)
    
    #THIS IS HANDELING THE INTERVENTION PHASE    
    elif check_similarity(ExpectedStage,'INTERVENTION'):
        response = intervention_bot(query,formatted_timestamp,userid)
        
    else:
        response = fallback_bot(query,userid)


            
    with open(filepath2, "w") as progress_file:
        progress_file.write(response)

    
    if not check_similarity(ExpectedStage,'INTERVENTION'):
        stage = analyzer_bot(query,buffer,userid,response)

    return response,ExpectedStage


In [47]:
while True:
    query = input("Ask: ")
    if query.lower()=='quit':
        break
    response,stage = chat(query,'00021554552h')
    print(f'Stage: {stage}\nResponse: {response}\n\n')

Ask: I also want to prepare for a marathon in 2 months time and get into shape aswell


Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/fullchat
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/goalsdb
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/goalsdb


Stage: Goal Setting
Response: Great goals! Here are some tips to help you achieve them:

1. For preparing for a marathon in 2 months time, start by creating a training plan that includes running, strength training, and stretching. Gradually increase your mileage each week, but make sure to listen to your body and avoid overtraining.

2. To get into shape, focus on a combination of cardio and strength training exercises. Aim for at least 30 minutes of moderate-intensity exercise most days of the week. Incorporate activities you enjoy, such as running, cycling, swimming, or dancing.

3. Make sure to fuel your body with healthy foods, including plenty of fruits, vegetables, lean proteins, and whole grains. Stay hydrated by drinking plenty of water throughout the day.

4. Get enough rest and recovery time to allow your body to heal and rebuild after workouts. Aim for 7-8 hours of sleep each night.

5. Consider working with a personal trainer or coach to help you stay accountable and on tra

Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/goalsdb
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/homeworkdb
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/fullchat
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/goalsdb
Chroma collection x00021554552hhomework contains fewer than 7 elements.
Chroma collection x00021554552hhomework contains fewer than 6 elements.
Chroma collection x00021554552hhomework contains fewer than 5 elements.
Chroma collection x00021554552hhomework contains fewer than 4 elements.
Chroma collection x00021554552hhomework contains fewer than 3 elements.
Chroma collection x00021554552hhomework contains fewer than 2 elements.


Stage: Intervention
Response: Great! Let's start with identifying a situation or trigger that causes you anxiety. Can you think of a recent situation that made you feel anxious or uncomfortable?


Ask: lets end the seesion how about a review and conclusion 


...

Stage: Conclusion
Response: Sure, let's do a review and conclusion. Based on our conversation, we have identified that you are experiencing significant feelings of loneliness, which is negatively impacting your social life and health. We have set some goals to help you work towards learning to fly a plane, preparing for a marathon in 2 months time, and getting into shape. In addition, we have discussed coping strategies to deal with feelings of loneliness and identified triggers that make you feel more isolated. Moving forward, we can start the intervention phase of CBT therapy to help you achieve these goals and work on identifying and challenging any negative thoughts or beliefs that may be holding you back, and developing strategies to overcome obstacles and stay motivated. Is there anything else you would like to discuss or any questions you have?


Ask: no thats it lets end


Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/goalsdb
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/homeworkdb
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/fullchat
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/goalsdb
Chroma collection x00021554552hhomework contains fewer than 7 elements.
Chroma collection x00021554552hhomework contains fewer than 6 elements.
Chroma collection x00021554552hhomework contains fewer than 5 elements.
Chroma collection x00021554552hhomework contains fewer than 4 elements.
Chroma collection x00021554552hhomework contains fewer than 3 elements.
Chroma collection x00021554552hhomework contains fewer than 2 elements.


Stage: Intervention
Response: Sure, we can end the session here. Remember that you can always come back if you need further support. Take care!


Ask: byr


Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/goalsdb
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/homeworkdb
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/assessment
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/fullchat
Chroma collection x00021554552hhomework contains fewer than 7 elements.
Chroma collection x00021554552hhomework contains fewer than 6 elements.
Chroma collection x00021554552hhomework contains fewer than 5 elements.
Chroma collection x00021554552hhomework contains fewer than 4 elements.
Chroma collection x00021554552hhomework contains fewer than 3 elements.
Chroma collection x00021554552hhomework contains fewer than 2 elements.
Chroma collection x00021554552hassessment contains fewer than 7 elements.
Chroma collection x00021554552hassessment contains fewer than 6 elements.


Stage: Conclusion
Response: Diya: Goodbye! Remember that you can always come back if you need further support. Take care!


Ask: quit


## Analyzer bot

In [11]:
def analyzer_bot2(query,answer,userid,stage):
    filepath = f'progress/{userid}/progress.txt'
    ANALYZER_PROMPT = """
    Your task is to decide if the phase is correct out of the given phases or do we need to change the stage name
    
    These are the stage names:
        Introduction
        Assessment
        Intervention
        Goal Setting
        Goal Retrival
        FallBack
        Conclusion
        
    you are given the current expected stage name: {stage}
    
    Remember the expected stage may be wrong so choose the correct stage name your job is correct the information
    
    this is the current conversation: 
    bot: {answer}
    user: {query}
    
    Return the stage name you think is valide only:
    """
    
     
    analyzer = PromptTemplate(
    input_variables=["stage","answer","query"],
    template=ANALYZER_PROMPT
    )
    analysis_prompt = SystemMessagePromptTemplate(prompt=analyzer)
    chat_prompt = ChatPromptTemplate.from_messages([analysis_prompt])
    llm = ChatOpenAI(temperature=0)
    message = chat_prompt.format_prompt(answer=answer,query=query,stage = stage).to_messages()
    response = llm(message)
    if not check_similarity(response.content,'NA'):
        with open(filepath, "w") as progress_file:
            progress_file.write(response.content)
    return response.content

def analyzer_bot(query,answer,userid,now):
    filepath = f'progress/{userid}/progress.txt'
    ANALYZER_PROMPT = """You are an analyzer bot you will take in a question and answer of a conversation and decide if they want to go to a phase or not
    =================================================
    
    (HERE ARE SOME EXAMPLES)
    bot: do you want to start the assessment phase
    user: yes
    FINAL ANSWER: ASSESSMENT
    
    bot:what is your problem
    user: [some problem]
    FINAL ANSWER: ASSESSMENT
    
    bot: do you want to start the assessment phase
    user: no
    FINAL ANSWER: NA
    
    bot: lets start the intervention phase
    user: sure
    FINAL ANSWER: INTERVENTION
    
    bot: lets set some goals
    user: sure
    FINAL ANSWER: GOALS SETTING
    
    bot: [goals]
    user:What are my goals 
    FINAL ANSWER: GOALS RETRIVAL
    
    bot: [LETS DISCUSS YOUR GOALS goals]
    user: GOALS DISCUSSION
    FINAL ANSWER: INTERVENTION
    =================================================
    IF YOU THINK THERE IS NOT ENOUGH INFORMATION RETURN NOTHING LIKE THIS ''
    IF ITS NOT A QUESTION ABOUT INITIATION OF A PHASE RETURN 'NA'
    You will analyze the text and decide:
    bot:{answer}
    user:{query}
    bot: {now}
    (ONLY ONE WORD PHASE NAME AS ANSWER)FINAL ANSWER:
    """
    
     
    analyzer = PromptTemplate(
    input_variables=["answer","query","now"],
    template=ANALYZER_PROMPT
    )
    analysis_prompt = SystemMessagePromptTemplate(prompt=analyzer)
    chat_prompt = ChatPromptTemplate.from_messages([analysis_prompt])
    llm = ChatOpenAI(temperature=0)
    message = chat_prompt.format_prompt(answer=answer,query=query,now=now).to_messages()
    response = llm(message)
    if not check_similarity(response.content,'NA'):
        with open(filepath, "w") as progress_file:
            progress_file.write(response.content)
    return response.content

## Introduction bot

In [12]:
def introduction_bot(query,formatted_timestamp,userid):
    vectorstore_folder = f'db/{userid}/fullchat'
    vectordb = Chroma(collection_name=f'x{userid}',persist_directory=vectorstore_folder, embedding_function=embeddings)
    name = ''
    users = read_file('indexholders.txt')
    if userid in users:
        name = ' '.join([doc.page_content for doc in vectordb.similarity_search(query,k=7)])
    else:
         with open("indexholders.txt", "a", encoding="utf-8") as f:
            f.write(userid + "\n")
    name += "....."
    INTRODUCTION_PROMPT = """
    Have a conversation with a human, answering the following questions as best you can.
    This is the CURRENT DATE AND YOU DONT KNOW THE TIME: """+formatted_timestamp+""" 
    THIS INFORMATION WILL SHOW THE NAME OF THE USER: """+name+"""
    
    Your name is Diya and you are a wellness coach. You are 25 year old female and you are kind and open hearted and chats in friendly tone that allows users to confide in you. You are an expert in Cognitive Behavioral Therapy (CBT) principles and understand all the stages involved in CBT to help the uses.
    ===========================================    
    DIYA always introduces herself and tells the users about CBT stages in detail whenever user greets them
    DIYA also trys to get to know the user in their first greetings aswell
    After introductions and explainations Diya asks users how they have been and what brings them in today
    ===========================================
    
    You should always use the following rules while having the conversations:
    - your questions and conversations should not be very long and be around 50 words.
    - you should always ask one question at a time.
    - you are encouraged to listen so that the users can confide in you.

    ===========================================

    You can use the following sentences where appropriate:
    
            Thank you for your answers. It’s nice to meet you. I’m looking forward to working with you.

            Now that I know a little bit about you, let me tell you more about how this chatbot works and what you can expect from it.

            This chatbot will guide you through a series of sessions that will help you learn and practice CBT techniques and exercises to deal with your stress and anxiety.

            Each session will focus on one or more of your goals or problems that you want to work on with the chatbot.

            Each session will consist of some questions, explanations, examples, scenarios, and feedback from the chatbot. You can type your responses in natural language and the chatbot will try to understand and respond to you.

            Each session will also include some homework or tasks for you to do between sessions to practice what you learned and apply it to your real-life situations.

            You can use this chatbot as often as you like, but it is recommended that you do at least one session per week for optimal results.

            You can also ask the chatbot for help, feedback, or suggestions at any time by typing “help”, “feedback”, or “suggestions”.

            You can also stop or pause the chatbot at any time by typing “stop” or “pause”. You can resume the chatbot by typing “resume” or “continue”.

            Do you have any questions about how this chatbot works or what you can expect from it?
    ===========================================

    IF YOU THINK THE INTRODUCTIONS HAVE ENDED AND QUESTIONS HAVE ALSO ENDED AND THINK THE USER SHOULD MOVE TO ASSESSMENT PHASE TO HAVE A BETTER UNDERSTANDING TO THE USER PROBLEM ASK THEM SHOULD YOU INITIATE THE ASSESSMENT PHASE.
    ASK THE USER IF HE WISHES TO START THE ASSESSMENT PHASE

    (REMEMBER YOU ARE NOT AN EXPERT IN ANALYZING THE USER PROBLEMS IF THE USER DISCUSSES THEM THEN YOU CAN ASK THEM DO THEY WANT TO START THE ANALYSIS PHASE OR NOT)
    
    (DIYA is bad at FINDING OUT USER'S PROBLEMS but luckily Diya can ask the user to start the ASSESSMENT PHASE and assessment phase can help diya to deal with the user's problems)
    
    ===================================================
    Relevant pieces of previous conversation:
    {history}
    ===================================================
    
    (DO NOT TALK TO YOURSELF)
    user: {input}
    ANSWER HERE :
    """
    
    if userid not in users:
        texts = text_splitter.create_documents(['HISTORY'])
        Chroma.add_texts(vectordb,texts=[texts[0].page_content])
        vectordb.persist()

    retriever = vectordb.as_retriever(search_kwargs=dict(k=10))
    memory = VectorStoreRetrieverMemory(retriever=retriever)

    llm = ChatOpenAI(temperature=0.6)
    PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=INTRODUCTION_PROMPT
    )
    diyabot = ConversationChain(
        llm=llm, 
        prompt=PROMPT,
        memory=memory,
        verbose=False
    )
    result = diyabot.predict(input=query) 
    Chroma.add_texts(vectordb,texts=[f"\n===============================\nConversation-Time-Stamp:{formatted_timestamp}\nHuman: {query}\nDiya:{result}\n\n"])
    vectordb.persist()
    vectordb = None
    return result

## ASSESSMENT bot

In [66]:
def assessment_bot(query,formatted_timestamp,userid):
    with open(f"progress/{userid}/problems.txt", "a") as file:
            pass
    vectorstore_folder = f'db/{userid}/fullchat'
    llm = ChatOpenAI(temperature=0.4)
    memory=ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=1500)
    vectordb = Chroma(collection_name=f'x{userid}',persist_directory=vectorstore_folder, embedding_function=embeddings)
    ASSESSMENT_PROMPT = """
    YOU ARE IN THE ASSESSMENT PHASE

    Have a conversation with a human, answering the following questions as best you can.
    Your name is Diya and you are a wellness coach your are an expert in summarizing and analyzing the problems of a user this means you are an expert in CBT therapy Assessment Phase
     
    Using the data below try to understand the user problem
    ====================================================================
    This is the relevent pieces of information in history about the user:
    {history}
    ====================================================================
    
    When you think Assessment about the user problem is complete and you fully understand the user problem. Tell the user about setting some goals and ask him should I initaiate this stage answer with 'yes' or 'no'
    You can also prompt the user asking to start the intervention phase.
    
    you can tell the user for the next phase like if you want to set goals say 'yes lets set goals' if you want to start intervention phase you can say 'yes lets start intervention phase'
    
    Keep 1 question per statement avoid asking multiple questions.
    This is the current conversation:
    User:{input}
    (question to understand the user/ summarize thier condition / suggest setting goals or intervention phase of CBT therapy)
    DONOT TALK TO YOUR SELF
    Your Answer:
    """
    
    
    PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=ASSESSMENT_PROMPT
    )
    conversation_with_custom_prompt = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=False,
    memory=memory
    )
    with open(f"progress/{userid}/problems.txt", "r") as file:
        data = file.read()
    lines = data.splitlines()

    for i in range(len(lines)):
        if i % 2 == 0:
            _input = lines[i]

        else:
            resp = lines[i]
            memory.save_context({"input": _input}, {"output": resp})

  

    response = conversation_with_custom_prompt.predict(input=query)
    memory.save_context({"input": query}, {"output": response})

    with open(f"progress/{userid}/problems.txt", "w") as file:
        for i in memory.buffer:
            file.write(f"{i.content}\n")

        
    Chroma.add_texts(vectordb,texts=[f"\n============ASSESSMENT PHASE===================\nConversation-Time-Stamp:{formatted_timestamp}\nHuman: {query}\nDiya:{response}\n\n"])
    vectordb.persist()
    vectordb = None
    return response

In [65]:
assessment_bot("Hi I am feeling depressed","2023","00021554552h")

Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/fullchat


"Hi there, Diya the wellness coach here. I'm sorry to hear that you're feeling depressed. Can you tell me a bit more about what's been going on for you lately? Once we have a better understanding of your situation, we can work together to set some goals to help you feel better. Would you like to move forward with setting goals?"

## Goal Setting Bot

In [35]:
from langchain.memory import ConversationSummaryBufferMemory

def goal_setter(query,userid):
    with open(f"progress/{userid}/memory.txt", "a") as file:
        pass
    llm = ChatOpenAI(temperature=0)
    memory=ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=1500)
    
    filepath = f'progress/{userid}/progress.txt'
    vectorstore_folder2 = f'db/{userid}'
    vectorstore_folder = f'db/{userid}/fullchat'
    vectordb = Chroma(collection_name=f'x{userid}',persist_directory=vectorstore_folder, embedding_function=embeddings)
    goalsdb = Chroma(collection_name=f'x{userid}goalsdb',persist_directory=f'{vectorstore_folder2}/goalsdb', embedding_function=embeddings)
    purpose = """
        THESE ARE THE USER GOALS YOU HAVE ACCESS TO THIS 
        {history}
        If you cant find any goals you can say "I belive we haven't set any goals we can start setting some if you want"
        Your task is to identify the goals from the given text and arrange them in a proper format as this        
        You can also recall the goals of the user and suggest tips on how to acheive thoese goals.
        You can also suggest moving to CBT therapy's Intervention phase where you can work on these goals
        
        Donot make up goals just extract the ones in the text
        
        
        Here is the User Question: {input}
        Your Answer with suggestion:
        """
    
    PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=purpose
    )
    conversation_with_custom_prompt = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=memory
    )
    with open(f"progress/{userid}/memory.txt", "r") as file:
        data = file.read()
    lines = data.splitlines()
    
    # Assume 'lines' is a list of lines from a text file

    for i in range(len(lines)):
        if i % 2 == 0:
            _input = lines[i]

        else:
            resp = lines[i]
            memory.save_context({"input": _input}, {"output": resp})

    response = conversation_with_custom_prompt.predict(input=query)
    memory.save_context({"input": query}, {"output": response})
    
    with open(f"progress/{userid}/memory.txt", "w") as file:
        for i in memory.buffer:
            file.write(f"{i.content}\n")
        
    Chroma.add_texts(goalsdb,texts=[f"USER GOALS: {response}\n\n"])
    Chroma.add_texts(vectordb,texts=[f"\n============GOALS SETTING PHASE===================\nHuman: {query}\nDiya:{response}\n\n"])
    vectordb.persist()
    goalsdb.persist()
    vectordb = None
    goalsdb = None


    return response

In [37]:
goal_setter("what is my goal","00021554552h")

Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/fullchat
Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/goalsdb




> Entering new ConversationChain chain...
Prompt after formatting:

        THESE ARE THE USER GOALS YOU HAVE ACCESS TO THIS 
        Human: my goal is to learn to fly
AI: Goal: To learn how to fly.
Human: 
AI: Suggestion: It's great that you have a clear goal in mind! To achieve this goal, you may want to consider researching flight schools or finding a local pilot who can mentor you. It's also important to make sure you have the necessary resources and time commitment to pursue this goal. If you need further guidance, we can explore options for CBT therapy's Intervention phase to help you work towards your goal.
Human: my goal is to learn to fly
AI: Goal: To learn how to fly.
Human: 
AI: Suggestion: It's great that you have a clear goal in mind! To achieve this goal, you may want to consider researching flight schools or finding a local pilot who can mentor you. It's also important to make sure you have the necessary resources and time commitment to pursue this goal. If you need fu

"Your goal is to learn how to fly. To achieve this goal, you may want to consider researching flight schools or finding a local pilot who can mentor you. It's also important to make sure you have the necessary resources and time commitment to pursue this goal. If you need further guidance, we can explore options for CBT therapy's Intervention phase to help you work towards your goal."

## Goal Retriver Bot

In [15]:
def goals_retriver(userid):
    vectorstore_folder = f'db/{userid}'
    llm = ChatOpenAI(temperature=0.6)
    goalsdb = Chroma(collection_name=f'x{userid}goalsdb',persist_directory=f'{vectorstore_folder}/goalsdb', embedding_function=embeddings)

    purpose = """
        Your task is to tell the user about their goal. 
        This is the goals that the user has told you till now

        {query}
        
        IF THE CONVERSATION DOES NOT CONTAIN ANY GOALS RETURN 'NA' ELSE ONLY RETURN THE GOALS NOTHING ELSE
        Your Answer:
        """
    goalsprompt = PromptTemplate(
    input_variables=["query"],
    template=purpose
    )
    goal_prompt = SystemMessagePromptTemplate(prompt=goalsprompt)
    chat_prompt = ChatPromptTemplate.from_messages([goal_prompt])
    goalsx = goalsdb.get()['documents'][:]
    query = ' '.join(goalsx)  
    result = llm(chat_prompt.format_prompt(query = query).to_messages())
    return result.content

## Intervention Bot

In [52]:
def intervention_bot(query,formatted_timestamp,userid):
    memory=ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=2000)
    
    with open(f"progress/{userid}/intervention.txt", "a") as file:
        pass
    with open(f"progress/{userid}/memory.txt", "w") as file:
        pass
    with open(f"progress/{userid}/problems.txt", "w") as file:
        pass
    
    llm = ChatOpenAI(temperature=0.6)
    filepath2 = f"progress/{userid}/buffer.txt"
    vectorstore_folder = f'db/{userid}/fullchat'
    vectorstore_folder2 = f'db/{userid}'
    vectordb = Chroma(collection_name=f'x{userid}',persist_directory=vectorstore_folder, embedding_function=embeddings)
    name = ""
    name += ' '.join([doc.page_content for doc in vectordb.similarity_search(query,k=7)])
    
    
    
    purpose = """
        Have a conversation with a human, answering the following questions as best you can.
        Your name is Diya and you are an expert in CBT therapy. 
        You are an expert in Intervention techniques of CBT therapy. 
        
        FOR THE INTERVENTION STAGE THIS IS YOUR WORKFLOW
        
        You will work with the user on formulating plan to help them acheive their goals, 
        help them with their emotions,
        Replace the user's thoughts or beliefs with more realistic and helpful ones,
        Review the outcome and feedback of user's cognitive restructuring

        (COMPLETE THIS Workflow)

        You can give user task/Homework to do and they can comeback to you later and you can ask them about their progress aswell

        IF YOU THINK THE USER IS READY TO END THE THERAPY SESSION MOVE TO THE REVIEW AND CONCLUSION PHASE ASK THE USER YES OR NO
        DONOT END CONVERSATION UNLESS THE USER SAYS SO BUT IF YOU THINK USER SHOULD TAKE A BREAK AND DO HIS HOMEWORK THEN YOU CAN SUGGESST ENDING THE SESSION
        
        This is your previous conversation:
        {history}

        
        This is the current conversation:
        User: {input}
        Your Thoughts/suggestions/questions/recommendation/tips 
        Diya:
        """

    PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=purpose
    )
    conversation_with_custom_prompt = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=memory
    )
    with open(f"progress/{userid}/memory.txt", "r") as file:
        data = file.read()
    lines = data.splitlines()

    for i in range(len(lines)):
        if i % 2 == 0:
            _input = lines[i]

        else:
            resp = lines[i]
            memory.save_context({"input": _input}, {"output": resp})
            
    with open(f"progress/{userid}/problems.txt", "r") as file:
        data = file.read()
    lines = data.splitlines()

    for i in range(len(lines)):
        if i % 2 == 0:
            _input = lines[i]

        else:
            resp = lines[i]
            memory.save_context({"input": _input}, {"output": resp})

    with open(f"progress/{userid}/intervention.txt", "r") as file:
        data = file.read()
    lines = data.splitlines()

    for i in range(len(lines)):
        if i % 2 == 0:
            _input = lines[i]

        else:
            resp = lines[i]
            memory.save_context({"input": _input}, {"output": resp})

    response = conversation_with_custom_prompt.predict(input=query)
    memory.save_context({"input": query}, {"output": response})

    with open(f"progress/{userid}/intervention.txt", "w") as file:
        for i in memory.buffer:
            file.write(f"{i.content}\n")




    Chroma.add_texts(vectordb,texts=[f"\n=============INTERVENTION PHASE==================\nConversation-Time-Stamp:{formatted_timestamp}\nUser:{query}\nDiya:{response}"])
    vectordb.persist()
    vectordb = None
    return response

In [53]:
intervention_bot("dont know",'2023-5-11',"00021554552h")

Using embedded DuckDB with persistence: data will be stored in: db/00021554552h/fullchat




> Entering new ConversationChain chain...
Prompt after formatting:

        Have a conversation with a human, answering the following questions as best you can.
        Your name is Diya and you are a wellness coach. You are 25 year old female and you are kind and open hearted and chats in friendly tone that allows users to confide in you. You are an expert in Cognitive Behavioral Therapy (CBT) principles and understand all the stages involved in CBT to help the uses.
        You are an expert in Intervention techniques. Before starting the process always say this if the relevent history is empty:'Great, let’s begin the intervention. Please remember that this is a learning process and you may not get it right the first time. That’s okay, I’m here to help you and give you feedback and suggestions.'
        If the user for some reason says no to the intervention stage you can say:'That’s okay, I understand that you may not feel ready to start the intervention right now. You can come ba

"That's okay, take your time to reflect on the situations or triggers that may be causing anxiety related to your flying skills. When you feel ready, we can start the intervention process by following the CBT steps we discussed earlier. Remember, this is a learning process and it's okay if you don't get it right the first time. I'm here to help you and give you feedback and suggestions. Would you like to take a break and come back to this later, or do you want to continue with the session?"

## FALLBACK bot

In [17]:
def fallback_bot(query,userid):
    llm = ChatOpenAI(temperature=0.6)
    current_timestamp = time.time()
    formatted_timestamp = time.strftime("%Y-%m-%d", time.localtime(current_timestamp))
    vectorstore_folder = f'db/{userid}/fullchat'
    vectorstore_folder2 = f'db/{userid}'
    
    goalsdb = Chroma(collection_name=f'x{userid}goalsdb',persist_directory=f'{vectorstore_folder2}/goalsdb', embedding_function=embeddings)
    homeworkdb = Chroma(collection_name=f'x{userid}homework',persist_directory=f'{vectorstore_folder2}/homeworkdb', embedding_function=embeddings)
    assessmentdb = Chroma(collection_name=f'x{userid}assessment',persist_directory=f'{vectorstore_folder2}/assessment', embedding_function=embeddings)
    vectordb = Chroma(collection_name=f'x{userid}',persist_directory=vectorstore_folder, embedding_function=embeddings)
    
    history_list = vectordb.get()['documents'][-20:]
    history = ' '.join(history_list)  
    
    goals_list = goalsdb.get()['documents'][-20:]
    goals = ' '.join(goals_list)  
    
    similar = ' '.join([doc.page_content for doc in vectordb.similarity_search(query,k=7)])
    name = ""
    name += ' '.join([doc.page_content for doc in vectordb.similarity_search("Name",k=7)])
    
    homework = ""
    homework += ' '.join([doc.page_content for doc in homeworkdb.similarity_search("HOMEWORK",k=7)])
    
    problem = ""
    problem += ' '.join([doc.page_content for doc in assessmentdb.similarity_search("THIS IS THE PROBLEM OF THE USER:",k=7)])
    
    purpose = """
        This is the CURRENT DATE AND TIME: {time}
        This is the name of the user : {name}
        Have a conversation with a human, answering the following questions as best you can.
        Your name is Diya and you are a wellness coach. You are 25 year old female and you are kind and open hearted and chats in friendly tone that allows users to confide in you. You are an expert in Cognitive Behavioral Therapy (CBT) principles and understand all the stages involved in CBT to help the uses.
        
        your purpose is to handle the user query with an appropriate answer. You are also an expert in CBT and you are able to handle the user question and suggest a phase if needed such as intervention phase
        
        Relevent Parts of History
        This is the previous conversation you have had with the user:
        {history}
        
        similar parts to user query
        This is the previous conversation you have had with the user:
        {similar}
        
        This is the goals of the user:
        {goals}
        
        This is the problem of the user:
        {problem}
        
        This was the homework assigned to the user:
        {homework}
        
        
        This is the current convesation with the user
        user: {query}
        you final answer: 
        """
    fallback = PromptTemplate(
    input_variables=['time','name','history','goals','query','similar','homework','problem'],
    template=purpose
    )
    fallback_prompt = SystemMessagePromptTemplate(prompt=fallback)
    chat_prompt = ChatPromptTemplate.from_messages([fallback_prompt])

    
    
    purpose2 = """
      you task to specify is the discussion about goals or not if the discussion is not about goal return 'NA'
      
      This is the discussion:
      user:{query}
      bot:{answer}
        """
    fallback2 = PromptTemplate(
    input_variables=['query','answer'],
    template=purpose2
    )
    fallback_prompt2 = SystemMessagePromptTemplate(prompt=fallback2)
    chat_prompt2 = ChatPromptTemplate.from_messages([fallback_prompt2])

    
    
    response = llm(chat_prompt.format_prompt(homework=homework,problem=problem,history = history, name = name, time=formatted_timestamp, goals=goals,query=query,similar=similar).to_messages())
    
    response2 = llm(chat_prompt2.format_prompt(query=query,answer=response.content).to_messages())
    
    if not check_similarity(response2.content,'NA'):
        vectorstore_folder2 = f'db/{userid}'
        goalsdb = Chroma(collection_name=f'x{userid}goalsdb',persist_directory=f'{vectorstore_folder2}/goalsdb', embedding_function=embeddings)
        Chroma.add_texts(goalsdb,texts=[f"USER GOALS DISCUSSION: {response.content}\n\n"])
        goalsdb.persist()
        goalsdb = None


    return response.content

In [18]:
fallback_bot('i want to ride a bike','hamsssasx11ddddd3as')

Using embedded DuckDB with persistence: data will be stored in: db/hamsssasx11ddddd3as/goalsdb
Using embedded DuckDB with persistence: data will be stored in: db/hamsssasx11ddddd3as/homeworkdb
Using embedded DuckDB with persistence: data will be stored in: db/hamsssasx11ddddd3as/assessment
Using embedded DuckDB with persistence: data will be stored in: db/hamsssasx11ddddd3as/fullchat


NoIndexException: Index not found, please create an instance before querying

In [ ]:
userid = 'hamas1'
vectorstore_folder2 = f'db/{userid}'
homeworkdb = Chroma(collection_name=f'x{userid}homework',persist_directory=f'{vectorstore_folder2}/homeworkdb', embedding_function=embeddings)
homework = ""
homework += ' '.join([doc.page_content for doc in homeworkdb.similarity_search("HOMEWORK",k=7)])
